In [2]:
import pandas as pd
import numpy as np
import re
import os
import glob

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

In [3]:
dataset = pd.read_csv('./DiseaseSymptomKB.csv', encoding='utf-8', index_col=None, header=0)

In [4]:
symptom_dummies = pd.get_dummies(dataset.Symptom)
diseases = dataset['Disease']
sd_pivoted = pd.concat([diseases,symptom_dummies], axis=1)

In [5]:
symptom_dummies

,C0000727,C0000731,C0000737,C0002416,C0002962,C0003123,C0003126,C0003862,C0003962,C0004093,...,C1305739,C1313921,C1321756,C1384489,C1384606,C1405524,C1444773,C1511606,C1513183,C1517205
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
symptoms = sd_pivoted.columns[1:]

In [7]:
sd_pivoted = sd_pivoted.groupby('Disease').sum()

In [8]:
sd_pivoted

,C0000727,C0000731,C0000737,C0002416,C0002962,C0003123,C0003126,C0003862,C0003962,C0004093,...,C1305739,C1313921,C1321756,C1384489,C1384606,C1405524,C1444773,C1511606,C1513183,C1517205
Disease,,,,,,,,,,,,,,,,,,,,,
C0001175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0001418,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0001511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0001973,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0002395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0002871,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
C0002895,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0003507,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C0003537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sd_pivoted = sd_pivoted.reset_index()

In [ ]:
x = sd_pivoted[symptoms]

In [ ]:
y = sd_pivoted['Disease']

In [ ]:
mnb = MultinomialNB()
mnb = mnb.fit(x, y)

mnb.score(x, y)

In [ ]:
disease_pred = mnb.predict(x)

In [ ]:
disease_pred

In [ ]:
joblib.dump(mnb,'model.pkl', protocol=2)

In [ ]:
features = symptoms.tolist()

In [ ]:
features

In [ ]:
f= open("features.txt","w+")

In [ ]:
f.write(','.join(features))

In [ ]:
f.close()